# 电商评分预测 - PyCaret多分类实现

> 🎯 将评分1-5视为5个类别，使用分类方法预测

---

In [ ]:
# 环境准备
from pycaret.classification import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 加载数据（使用原项目数据）
data_path = "../../../supervised_learning/projects/04_ecommerce_rating_prediction/data/Womens_Clothing_E-Commerce_Reviews.csv"
data = pd.read_csv(data_path)

print(f"数据形状: {data.shape}")
data.head()

In [ ]:
# 数据预处理
# 1. 删除无用列
data = data.drop(['Unnamed: 0', 'Title', 'Review Text', 'Positive Feedback Count'], axis=1, errors='ignore')

# 2. 处理缺失值
data = data.dropna(subset=['Rating'])

# 3. 将Rating转为字符串（分类任务）
data['Rating'] = data['Rating'].astype(int).astype(str)

# 4. 查看评分分布
print("\n评分分布:")
print(data['Rating'].value_counts().sort_index())

print(f"\n清洗后数据形状: {data.shape}")

## PyCaret初始化

In [ ]:
# 初始化多分类环境
clf = setup(
    data=data,
    target='Rating',
    session_id=42,
    
    # 数据预处理
    normalize=True,
    fix_imbalance=True,          # 处理类别不平衡（重要！）
    
    # 特征工程
    feature_selection=True,
    
    # 评估设置
    fold=5,                      # 5折交叉验证
    verbose=True
)

## 模型对比

In [ ]:
# 对比模型（多分类）
best_model = compare_models(
    sort='Accuracy',
    n_select=1
)

In [ ]:
# 调优
tuned_model = tune_model(
    best_model,
    optimize='Accuracy',
    n_iter=30
)

## 可视化

In [ ]:
# 混淆矩阵
plot_model(tuned_model, plot='confusion_matrix')

In [ ]:
# 类别报告
plot_model(tuned_model, plot='class_report')

In [ ]:
# 特征重要性
plot_model(tuned_model, plot='feature')

## 评估

In [ ]:
# 测试集预测
predictions = predict_model(tuned_model)

from sklearn.metrics import accuracy_score, classification_report, mean_absolute_error

y_test = predictions['Rating'].astype(int)
y_pred = predictions['prediction_label'].astype(int)

accuracy = accuracy_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print(f"MAE: {mae:.4f}")
print(f"\n分类报告:")
print(classification_report(y_test, y_pred))

## 总结

### 多分类方法特点

**优势**:
- ✅ 直接预测评分类别
- ✅ 可以使用分类评估指标
- ✅ AutoML自动处理类别不平衡

**劣势**:
- ❌ 忽略评分顺序（3星>1星）
- ❌ 预测错误代价不一致

**下一步**: 查看 `02_pycaret_regression.ipynb` 对比回归方法